In [41]:
import spacy
from scipy import spatial

nlp = spacy.load('en_core_web_md')
cosine_similarity = lambda vec1,vec2: 1 - spatial.distance.cosine(vec1,vec2)

def suggest_tags (text):
    
    #list of suggested tags
    
    suggested = []
    
    doc = nlp(text)
    
    #get the root word for the token ( lemmatization )
    #get rid of stop words and punctuations
    suggested = [token.lemma_ for token in doc if token.text not in nlp.Defaults.stop_words and token.text.isalpha()]

    #recheck for stop words and common words like 'is','be' etc...
    suggested = [word for word in suggested if word not in nlp.Defaults.stop_words and len(word)>3]
    if '-PRON-' in suggested :
        suggested.remove('-PRON-')
        
    #get entities form the doc object
    #get similar words for entities using vectorization and computing cosine diff
    sim_ent = []
    
    for ent in doc.ents:
        suggested.append(ent.text)
        vec1 = nlp.vocab[ent.text].vector
        if (vec1.sum()):
            for word in nlp.vocab:
                if word.has_vector:
                    if word.is_lower:
                        if word.is_alpha:
                            cs = cosine_similarity(vec1,word.vector)
                            sim_ent.append((word,cs))
    sim_ent = sorted(sim_ent,key=lambda item: -item[1])
    sim_ent = [t[0].text for t in sim_ent[:3]]
    if len(sim_ent)>0:
        for word in sim_ent:
            suggested.append(word)
            
    suggested = [word.lower() for word in suggested]
    
        
    #get all the CAPITALIZED words 
    for token in doc:
        if token.text.isupper():
            suggested.append(token.text)
            
    
    
    return list(set(suggested))

In [56]:
tags = suggest_tags("Is Narendra Modi really transforming India?")

In [57]:
tags

['nagpur',
 'chennai',
 'modi',
 'india',
 'narendra',
 'pvt',
 'transform',
 'narendra modi']